<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2025-01-15 08:36:17--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  18.4MB/s    in 14s     

2025-01-15 08:36:32 (17.7 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [2]:
!unzip -q concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [3]:
%%capture
!pip install tensorflow keras matplotlib pandas

In [4]:
import os
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt

In [5]:
def clean_ds_store_and_count_classes(data_dir):

    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file == '.DS_Store':
                os.remove(os.path.join(root, file))

    num_classes = len(os.listdir(data_dir))
    return num_classes

test_data_dir = '/content/concrete_data_week4/test'

num_classes_in_test = clean_ds_store_and_count_classes(test_data_dir)
print(f"Number of classes in test data after cleaning: {num_classes_in_test}")

Number of classes in test data after cleaning: 2


In [6]:
params = {
    'image_size': 224,
    'batch_size': 100,
    'num_epochs': 2,
    'class_mode': 'categorical',
    'num_classes': 2,
    'shuffle': True}

def batch_generator(path, preprocess_input, params):
    data_generator = ImageDataGenerator(
        preprocessing_function=preprocess_input)

    generator = data_generator.flow_from_directory(path,
        target_size=(params['image_size'],
                     params['image_size']),
        batch_size=params['batch_size'],
        class_mode=params['class_mode'],
        shuffle=params['shuffle'])

    return generator

In [7]:
train_generator = batch_generator(
    'concrete_data_week4/train',
    preprocess_input,
    params)
validation_generator = batch_generator(
    'concrete_data_week4/valid',
    preprocess_input,
    params)

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [8]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(params['num_classes'], activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=params['num_epochs']
)

Epoch 1/2


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


301/301 ━━━━━━━━━━━━━━━━━━━━ 595s 2s/step - accuracy: 0.7821 - loss: 10.8088 - val_accuracy: 0.9928 - val_loss: 0.0205
Epoch 2/2
301/301 ━━━━━━━━━━━━━━━━━━━━ 496s 2s/step - accuracy: 0.9940 - loss: 0.0197 - val_accuracy: 0.9936 - val_loss: 0.0166


In [10]:
model.save('classifier_vgg_model.keras')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model.
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [11]:
from keras.models import load_model

resnet = load_model('/content/classifier_resnet_model.keras')
vgg16 = load_model('/content/classifier_vgg_model.keras')
print(resnet.summary())
print(vgg16.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 4 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 2048)                │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2)                   │           4,098 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,595,910 (90.01 MB)

 Trainable params: 4,098 (16.01 KB)

...

 Total params: 63,413,960 (241.91 MB)

 Trainable params: 21,137,986 (80.64 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 42,275,974 (161.27 MB)

None


In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet50_input
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_vgg16_input

params = {
    'batch_size': 32,
    'shuffle': False,
    'target_size': (224, 224)
}

test_resnet_gen = ImageDataGenerator(preprocessing_function=preprocess_resnet50_input)
test_resnet_generator = test_resnet_gen.flow_from_directory(
    '/content/concrete_data_week4/test',
    target_size=params['target_size'],
    batch_size=params['batch_size'],
    shuffle=params['shuffle']
)

test_vgg16_gen = ImageDataGenerator(preprocessing_function=preprocess_vgg16_input)
test_vgg16_generator = test_vgg16_gen.flow_from_directory(
    '/content/concrete_data_week4/test',
    target_size=params['target_size'],
    batch_size=params['batch_size'],
    shuffle=params['shuffle']
)

Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [13]:
resnet_performance = resnet.evaluate(test_resnet_generator, verbose=1)

16/16 ━━━━━━━━━━━━━━━━━━━━ 13s 301ms/step - accuracy: 0.9990 - loss: 0.0104


In [14]:
vgg16_performance = vgg16.evaluate(test_vgg16_generator, verbose=1)

16/16 ━━━━━━━━━━━━━━━━━━━━ 28s 885ms/step - accuracy: 1.0000 - loss: 0.0078


In [15]:
print(f"ResNet performance: {resnet_performance}")
print(f"VGG16 performance: {vgg16_performance}")

ResNet performance: [0.011508909985423088, 0.9980000257492065]
VGG16 performance: [0.006493033841252327, 1.0]


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li>
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [16]:
import numpy as np

In [24]:
predictions_vgg16 = np.argmax(model.predict(test_vgg16_generator, verbose=1), axis=1)
predictions_resnet = np.argmax(model.predict(test_resnet_generator, verbose=1), axis=1)

print('Model VGG16 Predictions (first 5 images):')
for i in range(5):
    if predictions_vgg16[i] == 0:
        print(f'Image {i+1}, Prediction: Negative')
    else:
        print(f'Image {i+1}, Prediction: Positive')

print('Model ResNet50 Predictions (first 5 images):')
for i in range(5):
    if predictions_resnet[i] == 0:
        print(f'Image {i+1}, Prediction: Negative')
    else:
        print(f'Image {i+1}, Prediction: Positive')

16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 121ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 114ms/step
Model VGG16 Predictions (first 5 images):
Image 1, Prediction: Negative
Image 2, Prediction: Negative
Image 3, Prediction: Negative
Image 4, Prediction: Negative
Image 5, Prediction: Negative
Model ResNet50 Predictions (first 5 images):
Image 1, Prediction: Negative
Image 2, Prediction: Negative
Image 3, Prediction: Negative
Image 4, Prediction: Negative
Image 5, Prediction: Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).